<a href="https://colab.research.google.com/github/anjelammcgraw/Evaluating-Models-with-Eletheur-AI-s-Evaluation-Harness/blob/main/5_2_Model_Evaluation_Instruct_tuned_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ⚠ IMPORTANT ⚠

Please ensure your Colab runtime is set to the following:

A100 GPU

# Evaluating an Instruct-tuned Model

## Instruct-tuned Evaluation

We will now repeat the process we used on our baseline - but using the instruct-tuned version of our model!

### Load Mistral AI's Mistral-7B in 4-bit Quantization


In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 12.4 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

####❓ Question:

Taking a look at the [model card](https://huggingface.co/mistralai/Mistral-7B-v0.1) (and the linked resources on the card) is this an instruct-tuned model or not?

**ANSWER:** The model card for "mistralai/Mistral-7B-Instruct-v0.2" indicates that it **IS** a instruct-tuned model.

### Collect and Load the Eleuther AI Evaluation Harness

In [ ]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness
%cd lm-evaluation-harness
!pip install -e .

In [ ]:
import lm_eval
from lm_eval.models.huggingface import HFLM
eval_model = HFLM(model, batch_size=16)

In [ ]:
lm_eval.tasks.initialize_tasks()

In [ ]:
results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["hellaswag", "arc_easy"],
    num_fewshot=0,
    batch_size=16,
)

In [ ]:
import pandas as pd

pd.DataFrame(results["results"])

,hellaswag,arc_easy
"acc,none",0.655845,0.819444
"acc_stderr,none",0.004741,0.007893
"acc_norm,none",0.833201,0.771044
"acc_norm_stderr,none",0.00372,0.008622
alias,hellaswag,arc_easy


### Few-shot MMLU (Machine Learning)

In [ ]:
fs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=5,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 448/448 [00:34<00:00, 12.94it/s]


In [ ]:
import pandas as pd

pd.DataFrame(fs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.482143
"acc_norm,none",0.482143
"acc_norm_stderr,none",0.047428
"acc_stderr,none",0.047428
alias,mmlu_flan_n_shot_loglikelihood_machine_learning


### Zero-Shot MMLU (Machine Learning)

In [ ]:
zs_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_n_shot_loglikelihood_machine_learning"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running loglikelihood requests
100%|██████████| 448/448 [00:05<00:00, 85.45it/s] 


In [ ]:
import pandas as pd

pd.DataFrame(zs_mmlu_results["results"])

,mmlu_flan_n_shot_loglikelihood_machine_learning
"acc,none",0.482143
"acc_norm,none",0.482143
"acc_norm_stderr,none",0.047428
"acc_stderr,none",0.047428
alias,mmlu_flan_n_shot_loglikelihood_machine_learning


In [ ]:
cot_mmlu_results = lm_eval.simple_evaluate(
    model=eval_model,
    tasks=["mmlu_flan_cot_zeroshot_conceptual_physics"],
    num_fewshot=0,
    batch_size=16,
)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for hails/mmlu_no_train contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hails/mmlu_no_train
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
INFO:lm-eval:num_fewshot has been set to 0 for mmlu_flan_cot_zeroshot_conceptual_physics in its config. Manual configuration will be ignored.
INFO:lm-eval:Building contexts for task on rank 0...
INFO:lm-eval:Running generate_until requests

  0%|          | 0/26 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generati

In [ ]:
import pandas as pd

pd.DataFrame(cot_mmlu_results["results"])

,mmlu_flan_cot_zeroshot_conceptual_physics
alias,mmlu_flan_cot_zeroshot_conceptual_physics
"exact_match,get-answer",0.153846
"exact_match_stderr,get-answer",0.07216


####❓Question:

What *exactly* are these two benchmarks measuring?

**Few-Shot MMLU (Machine Learning):**

fs_mmlu_results: This evaluates the model's ability in a few-shot learning scenario. Few-shot learning measure model's capacity to understand and perform tasks with only a small number of examples provided as context.
The task mmlu_flan_n_shot_loglikelihood_machine_learning tests the model on its log-likelihood performance in machine learning-related questions, given a context of 5 examples (as indicated by num_fewshot=5).

**Zero-Shot MMLU (Machine Learning):**

zs_mmlu_results: This benchmark tests the model's zero-shot learning abilities, where the model is evaluated on tasks without any prior examples. Zero-shot learning tests the inherent knowledge and reasoning capabilities of the model.
This benchmark assesses how well the model can answer machine learning-related questions based solely on its pre-training knowledge and reasoning skills.
The task mmlu_flan_cot_zeroshot_conceptual_physics tests the model on  its ability to generate a coherent chain of thought leading to the answer, in the domain of conceptual physics, without any provided examples.